# Scikit-learn example: 1. Recursive feature elimination with cross-validation

Exemple original: <br>
https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py

Dans ce notebook, on teste un exemple de scikit-learn sur notre jeu de données. Nous allons faire une séléction de variables pour déterminer, parmi les 54 variables disponibles, 20 variables qui ont plus d'importance dans la prédiction de la variable *grav*, qui indique la gravité de l'accident.

## Analyse et preprocessing du jeu de données

### Import des données

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/20210317-145928/accidents-corp-2019.csv')

### Preprocessing

Les résultats du `Pandas Profiling` nous guident dans le preprocessing. <br>
1. On observe que le jeu de données a 7.3% de **valeurs manquantes** au total. Nous faisons le choix d'éliminer les colonnes contenant plus de 90% de valeurs manquantes.

In [3]:
data = data.drop(columns=[col for col in data.columns if data[col].isna().sum()/len(data)>0.9])

2. Le jeu de données contient des **variables** qui ne sont **pas exploitables** par le jeu de données, et que nous faisons le choix d'éliminer:
- les identifiants (comme la colonne *id_vehicule*)
- les colonnes à valeur constantes (colonne *an*)
- les colonnes à haute cardinalité, c'est à dire ayant un trop grand nombre de catégories (colonne *adr*). Pour en savoir plus sur la haute cardinalité: https://project.inria.fr/dirtydata/presentation-useful-results-from-dirtydata-for-machine-learning-in-python-on-non-curated-data/

In [4]:
unsupported_var = ['an','Num_Acc','id_vehicule','adr','com','pr','pr1','dep','voie','v1']
data = data.drop(columns=unsupported_var)

On modifie les colonnes *lat* et *long* pour avoir bien des valeures numériques:

In [5]:
data['lat'] = data['lat'].str.replace(',', '.', regex=True)
data['lat'] = data['lat'].astype(float)
data['long'] = data['long'].str.replace(',', '.', regex=True)
data['long'] = data['long'].astype(float)

Et on définit une nouvelle variable pour l'heure de l'accident, à partir de la variable *hrmn*:

In [6]:
def hour(n):
    n=str(n)
    if len(n)<4:
        n='0'+n
    return int(n[:2])

data['heure']=data['hrmn'].apply(hour)

In [7]:
data = data.drop(columns=['hrmn'])

Séléction et encoding de la target variable et des variables explicatives:

In [8]:
y = data['grav'].values
X = data.drop(columns=['grav'])

In [9]:
from sklearn import preprocessing

In [10]:
label_enc = preprocessing.LabelEncoder()

In [11]:
y = label_enc.fit_transform(y)

On fait du encoding des variables catégorielles. <br>
On fait le choix de faire du **CardinalEncoding**, pour éviter d'aumenter la dimension du jeux de données.

In [12]:
categorical_var = ['num_veh','place','catu','sexe','trajet','secu1','secu2','secu3','locp','actp',
                   'etatp','senc','catv','obs','obsm','choc','manv','motor','lum','agg','int','atm','col'
                  ,'catr','circ','vosp','prof','plan','surf','infra','situ']

In [13]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline

In [14]:
ordinal_enc = OrdinalEncoder(handle_unknown = 'error')

In [15]:
columns_trans =  make_column_transformer((ordinal_enc,categorical_var))

In [16]:
X = columns_trans.fit_transform(X)

## Recursive feature elimination

In [ ]:
# Original code: https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py
# Adapted by: Giulia Santarsieri <giulia.santarsieri@data.gouv.fr> for DGML
print(__doc__)

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV



# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications

min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy',
              min_features_to_select=min_features_to_select)
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

Automatically created module for IPython interactive environment


# Scikit-learn example: 2. Recursive feature elimination

In [ ]:
# Code adapted by: Giulia Santarsieri <giulia.santarsieri@data.gouv.fr> for DGML

print(__doc__)

from sklearn.svm import SVC
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

# Load the digits dataset
#digits = load_digits()
#X = digits.images.reshape((len(digits.images), -1))
#y = digits.target

# Create the RFE object and rank each pixel
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=20, step=1) #20 variables
rfe.fit(X, y)
#ranking = rfe.ranking_.reshape(digits.images[0].shape)
ranking = rfe.ranking_

# Plot pixel ranking
plt.matshow(ranking, cmap=plt.cm.Blues)
plt.colorbar()
plt.title("Ranking of pixels with RFE")
plt.show()

Automatically created module for IPython interactive environment
